___
# Insper - Ciência dos Dados - Projeto 1

# Obtenção das avaliações

Utilize este notebook para **baixar avaliações de livros**.

O código abaixo irá automativamente baixar um conjunto de avaliações reais de livros submetidas por usuários da Amazon.

Será gerado um arquivo `dados.xlsx` contendo as informações disponíveis para o projeto.

## Preparação do ambiente no jupyter

Vamos importar algumas bibliotecas e definir algumas funções úteis!

In [1]:
pip install urllib3


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import hashlib
from sklearn.utils import shuffle
import os.path
import urllib.request

In [21]:
def decode(byt):
    mask = b'dataset'
    lmask = len(mask)
    return bytes(c ^ mask[i % lmask] for i, c in enumerate(byt))

def get_data(usuario="user", avaliacoes=500):
    file_path = '../../data/dados.xlsx'
    if not os.path.isfile(file_path):

        print(f'Ok! Vou baixar avaliações de livros da Amazon!')
        print('\nAguarde... Este processo pode demorar alguns minutos!')

        target = decode(b'\x0c\x15\x00\x11\x00_[K\x13\x15\x16]\x02\x1d\x10\t\x01\x03\x06\x16\x11\x16\x02\x1b\x0f\x07\x00\x1a\x10O\x17\x0e\x1eJ\x18\x05\x13\x1d\x07\x16\t\x1d\x07\x08\x15\x0f\x12J\x16\x0b\x0e\x1f\x12^\x17\x11\x12\x08\x11\x16\x00H\x04\x0b\x13\x00\x14\x14\x10\x11\x17\x04[\x0c\x12\x16\x00\x01\x13[\x03\x1c\n\x1f\x17>\x04\x15,\x0b\x11\x03').decode('ascii')
        resource = urllib.request.urlopen(target)
        content =  resource.read().decode(resource.headers.get_content_charset())
        llines = content.split("\r\n")

        target = decode(b'\x0c\x15\x00\x11\x00_[K\x13\x15\x16]\x02\x1d\x10\t\x01\x03\x06\x16\x11\x16\x02\x1b\x0f\x07\x00\x1a\x10O\x17\x0e\x1eJ\x18\x05\x13\x1d\x07\x16\t\x1d\x07\x08\x15\x0f\x12J\x16\x0b\x0e\x1f\x12^\x17\x11\x12\x08\x11\x16\x00H\x04\x0b\x13\x00\x14\x14\x10\x11\x17\x04[\x0c\x12\x16\x00\x01\x13[\x03\x1c\n\x1f\x17>\x04\x15,\x15\x1b\x17').decode('ascii')
        resource = urllib.request.urlopen(target)
        content =  resource.read().decode(resource.headers.get_content_charset())
        rlines = content.split("\r\n")

        seed = abs(hash(usuario)) % (10 ** 8)
        np.random.seed(seed)
        perc = np.random.uniform(0.4,0.6)
        left = np.ceil(perc * avaliacoes)
        lser = pd.Series(np.random.choice(llines,int(left),replace=False))
        right= avaliacoes-left
        rser = pd.Series(np.random.choice(rlines,int(right),replace=False))
        perc_train=0.6

        l_treino = int(np.floor(left * perc_train))
        r_treino = int(np.floor(right * perc_train))

        ltrain = lser.iloc[:l_treino]
        ltest = lser.iloc[l_treino:]
        rtrain = rser.iloc[:r_treino]
        rtest = rser.iloc[r_treino:]

        train = ltrain.append(rtrain,ignore_index = True)
        test = ltest.append(rtest,ignore_index = True)

        train = shuffle(train).reset_index(drop=True)
        test = shuffle(test).reset_index(drop=True)

        writer = pd.ExcelWriter(file_path)
        
        dft = pd.DataFrame({'Avaliacao' : train})

        dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)
        
        dfc = pd.DataFrame({'Avaliacao' : test})

        dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

        writer.save()

        print('Processo finalizado!')
        print('\nGeramos um arquivo chamado dados.xlsx na pasta data, confira!')

    else:

        raise Exception('Arquivo dados.xlsx já existe na pasta data! Apague o arquivo caso queira gerar novamente!')


## Insira seu usuário Insper

In [22]:
nome_usuario=""

In [23]:
nome_usuario = nome_usuario.strip()
if len(nome_usuario) == 0:
    raise Exception('Você precisa inserir seu nome de usuário na variável usuario!')

## Defina a quantidade de avaliações

Defina a quantidade de avaliações a serem baixadas. Lembre-se que a quantidade mínima de avaliações depende da quantidade de categorias que serão avaliadas.

In [24]:
qtde_avaliacoes = 0

In [25]:
if qtde_avaliacoes <= 0:
    raise Exception('Você precisa definir a quantidade de avaliações!')

## Obtenção das avaliações

Execute a próxima célula. Ela irá construir uma base de dados personalizada de avaliações.

Este processo poderá demorar alguns minutos. Quando ele finalizar, será gerado um arquivo `dados.xlsx` com as avaliações!

Pontos de atenção:
- Confira se a quantidade de avaliações foi suficiente (conforme enunciado)

In [26]:
get_data(nome_usuario,qtde_avaliacoes)

Ok! Vou baixar avaliações de livros da Amazon!

Aguarde... Este processo pode demorar alguns minutos!
Processo finalizado!

Geramos um arquivo chamado dados.xlsx na pasta data, confira!
